Задача: поиск аспектов в текстах отзывов, разделение на предложения или смысловые части

Список аспектов для поиска:

- задания
- дз / домашняя работа
- стиль выступлений
- лекции
- практики
- препод / преподаватель
- пары
- тест
- зачёт
- презентации
- баллы

In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 57.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=f6f1b7f99fad34c174686766ce195ba36f300b073ead95ccc9f9dc1c0891c79b
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import re
import pandas as pd
import numpy as np
import nltk
import spacy
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
stopwords = set(stopwords.words("russian"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Загрузка данных

In [ ]:
all_df = pd.read_csv("dest/courses_dataset.csv", sep="\t", encoding="utf-8")

In [ ]:
all_df = all_df[all_df["rubrics"].str.contains('образование_отзывус', na=False)]

In [ ]:
all_df

,Unnamed: 0,text,rating,rubrics
0,0,"Был скучноват. Много практики, решение задач п...",3.0,образование_онлайн_курсы;образование_отзывус
1,1,Первые пол семестра - упражнения всякие сценич...,5.0,образование_онлайн_курсы;образование_отзывус
2,2,"Чисто основы основ 3д моделинга, для тех кто в...",4.0,образование_онлайн_курсы;образование_отзывус
3,3,"Электив просто песня, две презентации за весь ...",5.0,образование_онлайн_курсы;образование_отзывус
4,4,Прекрасный электив помогающий разобраться в се...,5.0,образование_онлайн_курсы;образование_отзывус
...,...,...,...,...
1091,1091,Электив интересный. Учат моделировать детали и...,4.5,образование_онлайн_курсы;образование_отзывус
1092,1092,"Электив был очень интересным, преподаватель зн...",5.0,образование_онлайн_курсы;образование_отзывус
1093,1093,Этот электив я выбрал с целью погрузиться в ос...,5.0,образование_онлайн_курсы;образование_отзывус
1094,1094,За весь свой период обучения я слышал множеств...,5.0,образование_онлайн_курсы;образование_отзывус


# Чистка текста и токенизация

In [ ]:
def clean_text(text: str):
    # Убираем ссылки
    clean = re.sub(u'(http|ftp|https):\/\/[^ ]+', '', text)
    # Убираем все неалфавитные символы кроме дефиса и апострофа
    clean = re.sub(u'[^а-я^А-Я^\w^\-^\']', ' ', clean)
    # Убираем тире
    clean = re.sub(u' - ', ' ', clean)
    # Убираем дубликаты пробелов
    clean = re.sub(u'\s+', ' ', clean)
    # Убираем пробелы в начале и в конце строки
    clean = clean.strip().lower()
    return clean

def get_sentences(text: str):
    # Токенизация
    return nltk.sent_tokenize(text, language="russian")

# Список аспектов

In [11]:
aspects_list = [
    "задания",
    "домашняя работа",
    "стиль выступления",
    "лекция",
    "практика",
    "преподаватель",
    "пара",
    "тест",
    "зачёт",
    "презентация",
    "балл",
]

# Методы выделения аспектов

In [ ]:
class MethodSubstring():

    def find_aspects(self, text: str):
        aspects = {}
        for aspect in aspects_list:
            aspects[aspect] = []
        # Разделение на предложения
        sentences = get_sentences(text)
        # Очистка
        sentences_words = []
        for sentence in sentences:
            words = clean_text(sentence).split(" ")
            words = [token for token in words if token not in stopwords]
            sentences_words.append(words)
        # Лемматизация
        morph = MorphAnalyzer()
        for sentence_idx in range(len(sentences_words)):
            lemmatized = [morph.parse(word)[0].normal_form for word in sentences_words[sentence_idx]]
            for aspect in aspects_list:
                if aspect in lemmatized:
                    aspects[aspect].append(sentences[sentence_idx])
        return aspects

    def process(self, text: str):
        return self.find_aspects(text)

import torch
from transformers import AutoTokenizer, AutoModel

class MethodSimilarity():

    def spacy_tokenizer(self, sentence):
        return self.spacy_nlp(sentence).vector

    def transformers_tokenizer(self, sentence: str) -> torch.Tensor:
        tokens = self.transformers_tokenizer(sentence, return_tensors='pt')
        vector = self.transformers_model(**tokens)[0].detach().squeeze()
        return torch.mean(vector, dim=0).numpy()

    def __init__(self, tokenizer="spacy"):
        if tokenizer == "spacy": # Очень плохо
            self.tokenizer = self.spacy_tokenizer
            self.spacy_nlp = spacy.load("ru_core_news_lg")
        elif tokenizer == "rubert": # Ультра-гипер-плохо
            self.tokenizer = self.transformers_tokenizer
            self.transformers_tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
            self.transformers_model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
        elif tokenizer == "distiluse": # Хорошо
            self.tokenizer = self.transformers_tokenizer
            self.transformers_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/distiluse-base-multilingual-cased-v2")
            self.transformers_model = AutoModel.from_pretrained("sentence-transformers/distiluse-base-multilingual-cased-v2")
        elif tokenizer == "rubert-tiny2": # Очень плохо
            self.tokenizer = self.transformers_tokenizer
            self.transformers_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
            self.transformers_model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
        elif tokenizer == "sbert-pq": # Средне
            self.tokenizer = self.transformers_tokenizer
            self.transformers_tokenizer = AutoTokenizer.from_pretrained("inkoziev/sbert_pq")
            self.transformers_model = AutoModel.from_pretrained("inkoziev/sbert_pq")
        elif tokenizer == "sbert-synonymy": # Плохо
            self.tokenizer = self.transformers_tokenizer
            self.transformers_tokenizer = AutoTokenizer.from_pretrained("inkoziev/sbert_synonymy")
            self.transformers_model = AutoModel.from_pretrained("inkoziev/sbert_synonymy")
        else:
            self.tokenizer = None
            raise Exception("Invalid tokenizer. Available: spacy, rubert")

    def calc_similarity(self, vector1, vector2):
        return np.dot(vector1, vector2) / \
               (np.linalg.norm(vector1) * np.linalg.norm(vector2))

    def find_aspects(self, text: str, min_similarity: int):
        aspects = {}
        for aspect in aspects_list:
            aspects[aspect] = []
        # Разделение на предложения
        sentences = get_sentences(text)
        # Схожесть
        for sentence in sentences:
            sentence_vector = self.tokenizer(sentence)
            similarities = [(aspect, self.calc_similarity(self.tokenizer(aspect), sentence_vector))
                              for aspect in aspects_list]
            similarities.sort(key=lambda x: x[1], reverse=True)
            if similarities[0][1] > min_similarity:
                aspects[similarities[0][0]].append(sentence)
        return aspects

    def process(self, text: str, min_similarity: int = 0.2):
        return self.find_aspects(text, min_similarity)

# Проверка методов

In [ ]:
method_substring = MethodSubstring()

In [ ]:
method_substring.process("Преподаватель нормальный мужик. А может и не нормальный.")

{'задания': [],
 'домашняя работа': [],
 'стиль выступления': [],
 'лекция': [],
 'практика': [],
 'преподаватель': ['Преподаватель нормальный мужик.'],
 'пара': [],
 'тест': [],
 'зачёт': [],
 'презентация': [],
 'балл': []}

In [ ]:
method_similarity = MethodSimilarity(tokenizer="distiluse")

In [ ]:
method_similarity.process("Преподаватель - старушка-божий одуванчик и вообще очень хороший человек. \
                           Стиль выступлений замечательный. Лекции были очень увлекательными! \
                           Тесты были очень сложными, поэтому надо хорошо готовиться! \
                           Домашки почти не давали, каеф, задания давали редко. \
                           Задания на парах были интересными. Очень сложно давались тесты.",
                          0.2)

{'задания': ['Задания на парах были интересными.'],
 'домашняя работа': ['Домашки почти не давали, каеф, задания давали редко.'],
 'стиль выступления': ['Стиль выступлений замечательный.'],
 'лекция': ['Лекции были очень увлекательными!'],
 'практика': [],
 'преподаватель': ['Преподаватель - старушка-божий одуванчик и вообще очень хороший человек.'],
 'пара': [],
 'тест': ['Тесты были очень сложными, поэтому надо хорошо готовиться!',
  'Очень сложно давались тесты.'],
 'зачёт': [],
 'презентация': [],
 'балл': []}

# Тест векторизации с помощью трансформеров

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
# model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/distiluse-base-multilingual-cased-v2")
model = AutoModel.from_pretrained("sentence-transformers/distiluse-base-multilingual-cased-v2")

In [ ]:
def encode(document: str) -> torch.Tensor:
    tokens = tokenizer(document, return_tensors='pt')
    vector = model(**tokens)[0].detach().squeeze()
    return torch.mean(vector, dim=0)

def calc_similarity(vector1, vector2):
    return np.dot(vector1, vector2) / \
           (np.linalg.norm(vector1) * np.linalg.norm(vector2))

In [ ]:
documents = [
    "I know how to drive",
    "That restaurant was not as good as the last movie I watched.",
    "I'm selling a used car in good condition",
    "Food was okay, the rest so so",
    "I love cats, but don't really like hyenas",
    "On the road, you must be careful",
]

test = encode(documents[0])

for sentence in documents:
    print(calc_similarity(test, encode(sentence)))


0.99999994
0.5757822
0.71039003
0.6852526
0.4807013
0.87246233


In [ ]:

documents = [
    "Рэп - музыка быдла и гопников, поющих про секс, наркотики и перестрелки.",
    "Джаз - музыка для души, несмотря на то, что придуман неграми.",
    "Пожалуйста, не кормите белых медведей с рук, если носите кольца или браслеты.",
    "Хип-хоп, попса, классическая инструментальная музыка - я слушаю всё.",
    "Кто я? Кто ты? Аянами Рей."
]

test = encode(documents[0])

for sentence in documents:
    print(calc_similarity(test, encode(sentence)))


0.99999994
0.6502613
0.3867934
0.65753275
0.44148263


# Pipeline

Загруза моделей

In [4]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=26f1f19800b23883aa37ba90db7f2cfe3bfd4629d48287d21ca8b289d8357d84
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [6]:
from sentence_transformers import SentenceTransformer

MODELS_PATH="ai_models/"

MODELS = [
    "sentence-transformers/distiluse-base-multilingual-cased-v2",
    "inkoziev/sbert_pq",
]

for model_name in MODELS:
    model = SentenceTransformer(model_name)
    model.save(MODELS_PATH+model_name)

.gitattributes:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.84k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/410 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [21]:
import nltk
import re
import numpy as np

nltk.download('punkt')

def clean_text(text: str):
    # Убираем ссылки
    clean = re.sub(u'(http|ftp|https):\/\/[^ ]+', '', text)
    # Убираем все неалфавитные символы кроме дефиса и апострофа
    clean = re.sub(u'[^а-я^А-Я^\w^\-^\']', ' ', clean)
    # Убираем тире
    clean = re.sub(u' - ', ' ', clean)
    # Убираем дубликаты пробелов
    clean = re.sub(u'\s+', ' ', clean)
    # Убираем пробелы в начале и в конце строки
    clean = clean.strip().lower()
    return clean

def get_sentences(text: str):
    # Токенизация
    return nltk.sent_tokenize(text, language="russian")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Выделение ключевых аспектов

In [22]:
import torch
from transformers import AutoTokenizer, AutoModel

class MethodSubstring():

    def find_aspects(self, text: str):
        aspects = {}
        for aspect in aspects_list:
            aspects[aspect] = []
        # Разделение на предложения
        sentences = get_sentences(text)
        # Очистка
        sentences_words = []
        for sentence in sentences:
            words = clean_text(sentence).split(" ")
            words = [token for token in words if token not in stopwords]
            sentences_words.append(words)
        # Лемматизация
        morph = MorphAnalyzer()
        for sentence_idx in range(len(sentences_words)):
            lemmatized = [morph.parse(word)[0].normal_form for word in sentences_words[sentence_idx]]
            for aspect in aspects_list:
                if aspect in lemmatized:
                    aspects[aspect].append(sentences[sentence_idx])
        return aspects

    def process(self, text: str):
        return self.find_aspects(text)

class MethodSimilarity():

    def transformers_tokenizer(self, sentence: str) -> torch.Tensor:
        tokens = self.transformers_tokenizer(sentence, return_tensors='pt')
        vector = self.transformers_model(**tokens)[0].detach().squeeze()
        return torch.mean(vector, dim=0).numpy()

    def __init__(self, tokenizer="distiluse"):
        if tokenizer == "distiluse": # Хорошо
            self.tokenizer = self.transformers_tokenizer
            self.transformers_tokenizer = AutoTokenizer.from_pretrained("ai_models/sentence-transformers/distiluse-base-multilingual-cased-v2", local_files_only=True)
            self.transformers_model = AutoModel.from_pretrained("ai_models/sentence-transformers/distiluse-base-multilingual-cased-v2", local_files_only=True)
        elif tokenizer == "sbert-pq": # Средне
            self.tokenizer = self.transformers_tokenizer
            self.transformers_tokenizer = AutoTokenizer.from_pretrained("ai_models/inkoziev/sbert_pq", local_files_only=True)
            self.transformers_model = AutoModel.from_pretrained("ai_models/inkoziev/sbert_pq", local_files_only=True)
        else:
            self.tokenizer = None
            raise Exception("Invalid tokenizer. Available: spacy, rubert")

    def calc_similarity(self, vector1, vector2):
        return np.dot(vector1, vector2) / \
               (np.linalg.norm(vector1) * np.linalg.norm(vector2))

    def find_aspects(self, text: str, min_similarity: int):
        aspects = {}
        for aspect in aspects_list:
            aspects[aspect] = []
        # Разделение на предложения
        sentences = get_sentences(text)
        # Схожесть
        for sentence in sentences:
            sentence_vector = self.tokenizer(sentence)
            similarities = [(aspect, self.calc_similarity(self.tokenizer(aspect), sentence_vector))
                              for aspect in aspects_list]
            similarities.sort(key=lambda x: x[1], reverse=True)
            if similarities[0][1] > min_similarity:
                aspects[similarities[0][0]].append(sentence)
        return aspects

    def process(self, text: str, min_similarity: int = 0.2):
        return self.find_aspects(text, min_similarity)

In [23]:
ms = MethodSimilarity()



In [24]:
ms.process("Задания дают не сложные")

{'задания': ['Задания дают не сложные'],
 'домашняя работа': [],
 'стиль выступления': [],
 'лекция': [],
 'практика': [],
 'преподаватель': [],
 'пара': [],
 'тест': [],
 'зачёт': [],
 'презентация': [],
 'балл': []}